In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 로드
restaurant_data = pd.read_csv("/content/drive/MyDrive/df_meal.csv", encoding='euc-kr')
cafe_data = pd.read_csv("/content/drive/MyDrive/DACOS/df_cafe.csv")
# 태그 옵션 정의
tag_options = {
    '식당': {
        '맛': ['음식이 맛있어요', '빵이 맛있어요'],
        '특별한 메뉴': ['특별한 메뉴가 있어요'],
        '메뉴 다양성': ['메뉴 구성이 알차요'],
        '신선도': ['재료가 신선해요', '잡내가 적어요', '고기 질이 좋아요'],
        '가성비': ['가성비가 좋아요', '양이 많아요'],
        '서비스': ['친절해요', '반찬이 잘 나와요', '음식이 빨리 나와요'],
        '청결': ['매장이 청결해요', '화장실이 깨끗해요'],
        '혼밥' : ['혼밥하기 좋아요']
    },
    '카페': {
        '커피': ['커피가 맛있어요'],
        '디저트': ['디저트가 맛있어요'],
        '청결': ['매장이 청결해요', '화장실이 깨끗해요'],
        '서비스': ['친절해요', '포장이 깔끔해요'],
        '분위기': ['아늑해요', '인테리어가 멋져요'],
        '공부하기 좋음음' : ['집중하기 좋아요', '좌석이 편해요', '매장이 넓어요']
    },
    '술집': {
        '맛': ['음식이 맛있어요'],
        '서비스': ['친절해요'],
        '다양한 술': ['술이 다양해요'],
        '분위기': ['음악이 좋아요'],
        '대화하기 좋음' : ['대화하기 좋아요']
    }
}

# 식당만 고정된 음식 종류
category_options = {
    '식당': ['한식', '중식', '일식', '양식', '육류', '아시안음식', '샐러드']
}

# 사용자 입력 처리 함수
def get_user_selection(prompt, choices):
    while True:
        print(prompt)
        for i, choice in enumerate(choices, 1):
            print(f"{i}. {choice}")
        try:
            selection = int(input("선택 번호를 입력하세요: ")) - 1
            if 0 <= selection < len(choices):
                return choices[selection]
            else:
                print(f"잘못된 입력입니다. 1부터 {len(choices)} 사이의 숫자를 입력하세요.")
        except ValueError:
            print("숫자를 입력하세요.")

# 사용자 입력
try:
    # 데이터 선택
    selected_category = get_user_selection(
        "데이터를 선택하세요 (1: 식당, 2: 카페):", ["식당", "카페"]
    )

    if selected_category == "식당":
        selected_data = restaurant_data
        # 식당은 음식 종류 선택
        selected_food_category = get_user_selection(
            "음식 종류를 선택하세요:", category_options["식당"]
        )
        print(f"선택된 음식 종류: {selected_food_category}")
        # 필터링
        filtered_data = selected_data[selected_data['FoodCategory'] == selected_food_category]
    else:
        selected_data = cafe_data
        filtered_data = selected_data  # 전체 데이터 사용
        selected_food_category = None  # 구분 없음

    # 태그 선택
    selected_tags = get_user_selection(
        "중요하게 생각하는 태그를 선택하세요:", list(tag_options[selected_category].keys())
    )
    print(f"선택된 태그: {selected_tags}")

except KeyError as e:
    print(f"선택지 처리 중 오류가 발생했습니다: {e}")
    raise

# 태그 기반 평균값 기준 필터링 함수
def rank_stores_by_threshold(data, selected_tags, tag_options):
    """
    선택한 태그를 만족하는 가게를 평균값 기준으로 필터링
    - data: 데이터프레임
    - selected_tags: 사용자가 선택한 태그 리스트
    - tag_options: 태그 매핑
    """
    # 선택한 태그의 실제 컬럼 이름 매핑
    valid_tags = []
    for tag in selected_tags:
        if tag in tag_options:
            valid_tags.extend(tag_options[tag])  # 선택한 태그의 모든 관련 컬럼 추가

    # 데이터프레임에 존재하는 태그만 필터링
    valid_tags = [t for t in valid_tags if t in data.columns]
    if not valid_tags:
        print(f"선택한 태그 {selected_tags}와 관련된 유효한 데이터가 없습니다.")
        return pd.DataFrame()

    # 유효한 태그들의 합계 계산
    data['Score'] = data[valid_tags].sum(axis=1)
    threshold = data['Score'].mean()  # 평균값 계산
    print(f"태그 리뷰 합계 평균값 기준 임계값: {threshold:.2f}")

    # 평균값 이상인 데이터 필터링
    filtered_data = data[data['Score'] >= threshold]
    return filtered_data

# 필터링된 결과 계산
ranked_results = rank_stores_by_threshold(filtered_data, selected_tags, tag_options[selected_category])

# 결과 출력
if not ranked_results.empty:
    print(f"평균값 기준 임계값 이상을 만족하는 가게:")
    print(ranked_results[['가게명', 'FoodCategory', 'Score']].head(10))  # 상위 10개 출력
else:
    print(f"선택한 태그 {selected_tags}에 대해 평균값 기준 이상 만족하는 가게가 없습니다.")

데이터를 선택하세요 (1: 식당, 2: 카페):
1. 식당
2. 카페
선택 번호를 입력하세요: 1
음식 종류를 선택하세요:
1. 한식
2. 중식
3. 일식
4. 양식
5. 육류
6. 아시안음식
7. 샐러드
선택 번호를 입력하세요: 1
선택된 음식 종류: 한식
중요하게 생각하는 태그를 선택하세요:
1. 맛
2. 특별한 메뉴
3. 메뉴 다양성
4. 신선도
5. 가성비
6. 서비스
7. 청결
8. 혼밥
선택 번호를 입력하세요: 1
선택된 태그: 맛
태그 리뷰 합계 평균값 기준 임계값: 426.47
평균값 기준 임계값 이상을 만족하는 가게:
           가게명 FoodCategory  Score
33  샤브로21 숙명여대           한식    688
38       만나칼국수           한식    428
48        한입소반           한식   3077
62        구복만두           한식   1332
63      한강로칼국수           한식    717
65    남산드럼통 본점           한식    451
70      오복함흥냉면           한식    598
79         남영탉           한식   1260


<ipython-input-4-82192ca2e051>:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Score'] = data[valid_tags].sum(axis=1)


In [5]:
# 상위 태그 컬럼 제외
if selected_category == "식당":
  additional_excluded_columns = [
      "음식이 맛있어요",
      '빵이 맛있어요'
      '특별한 메뉴가 있어요',
      '메뉴 구성이 알차요',
      '재료가 신선해요',
      '잡내가 적어요',
      '고기 질이 좋아요'
      "친절해요",
      "가성비가 좋아요",
      "양이 많아요",
      '반찬이 잘 나와요',
      '음식이 빨리 나와요'
      "매장이 청결해요",
      '화장실이 깨끗해요'
  ]
else :
  additional_excluded_columns = [
      '커피가 맛있어요',
      '디저트가 맛있어요',
      '매장이 청결해요',
      '화장실이 깨끗해요',
      '친절해요',
      '포장이 깔끔해요',
      '아늑해요',
      '인테리어가 멋져요'
    ]

df= ranked_results

  # 첫 두 개 컬럼과 마지막 컬럼 제외
excluded_columns = df.columns[:3].tolist() + df.columns[-2:].tolist()

# 최종 제외 리스트
final_excluded_columns = excluded_columns + additional_excluded_columns

# 나머지 컬럼 리스트
remaining_columns = [col for col in df.columns if col not in final_excluded_columns]
print("Remaining Columns:", remaining_columns)

Remaining Columns: ['혼밥하기 좋아요', '친절해요', '특별한 메뉴가 있어요', '매장이 넓어요', '매장이 청결해요', '인테리어가 멋져요', '특별한 날 가기 좋아요', '단체모임 하기 좋아요', '아늑해요', '음식이 빨리 나와요', '주차하기 편해요', '뷰가 좋아요', '음료가 맛있어요', '아이와 가기 좋아요', '반려동물과 가기 좋아요', '룸이 잘 되어있어요', '오래 머무르기 좋아요', '환기가 잘 돼요', '차분한 분위기에요', '코스요리가 알차요', '비싼 만큼 가치있어요', '대화하기 좋아요', '건강한 맛이에요', '음악이 좋아요', '좌석이 편해요', '사진이 잘 나와요', '현지 맛에 가까워요', '고기 질이 좋아요', '샐러드바가 잘 되어있어요', '혼술하기 좋아요', '향신료가 강하지 않아요', '야외공간이 멋져요', '컨셉이 독특해요', '포장이 깔끔해요', '술이 다양해요', '직접 잘 구워줘요', '커피가 맛있어요', '디저트가 맛있어요', '기본 안주가 좋아요', '집중하기 좋아요', '빵이 맛있어요', '태그 총합']


In [11]:
# Input 텍스트와 Remaining Columns 정의
input_txt = [input("기타 입력 사항을 입력하시오: ")]

기타 입력 사항을 입력하시오: 퓨전식 이색 요리


In [12]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# KoBERT 모델 및 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [13]:
# 문장 임베딩 생성 함수
def get_embeddings(text_list):
    inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # 평균 풀링 사용
    return embeddings

# Threshold 설정
threshold = 0.72

# Input 텍스트와 Remaining Columns 임베딩
input_embeddings = get_embeddings(input_txt)
remaining_embeddings = get_embeddings(remaining_columns)

# 유사도 계산
similarity_matrix = cosine_similarity(input_embeddings.numpy(), remaining_embeddings.numpy())

# 유사도 결과를 데이터프레임으로 변환
similarity_df2 = pd.DataFrame(
    similarity_matrix,
    index=input_txt,
    columns=remaining_columns
)

# high_similarity_pairs 구성 및 결과 저장
high_similarity_pairs = []
result_data = []

for input_text in similarity_df2.index:
    similar_columns = similarity_df2.loc[input_text][similarity_df2.loc[input_text] >= threshold]

    # 상위 5개 선택 (유사도 기준 내림차순)
    selected_tags = similar_columns.sort_values(ascending=False).head(5)

    # 선택된 태그가 있다면 리스트로 저장, 없다면 빈 리스트
    if len(selected_tags) > 0:
        tag_list = selected_tags.index.tolist()
    else:
        tag_list = []

    # high_similarity_pairs에 추가
    for column, similarity in selected_tags.items():
        high_similarity_pairs.append((input_text, column, similarity))

    # 결과 데이터 구성
    result_data.append({"입력 텍스트": input_text, "최종 선택 태그의 리스트": tag_list})

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_data)

# 출력
print(result_df)

      입력 텍스트 최종 선택 태그의 리스트
0  퓨전식 이색 요리   [코스요리가 알차요]


In [14]:
# 디버깅: result_df의 컬럼 확인
print("result_df 컬럼들:", result_df.columns)

# 각 input_txt에 대해 조건을 만족하는 가게별 유사도 점수 계산
final_results = []

for input_text in result_df["입력 텍스트"]:
    # 디버깅: 입력 텍스트 확인
    print(f"현재 처리 중: {input_text}")

    # 해당 input_txt의 tag_list 가져오기
    if "최종 선택 태그의 리스트" not in result_df.columns:
        raise KeyError("'최종 선택 태그의 리스트' 컬럼이 존재하지 않습니다. result_df 컬럼들을 확인하세요.")

    tag_list = result_df[result_df["입력 텍스트"] == input_text]["최종 선택 태그의 리스트"].values[0]

    # tag_list가 비어 있는 경우 처리
    if len(tag_list) == 0:
        final_results.append({
            "입력 텍스트": input_text,
            "추천 가게": [],
        })
        continue

    # 태그 수의 비율 계산 (1 / len(tag_list) * 태그 수)
    df["태그 비율"] = df[tag_list].sum(axis=1) / len(tag_list)

    # 조건에 맞는 가게 필터링 (태그 수 >= len(tag_list))
    filtered_df = df[df[tag_list].gt(0).sum(axis=1) >= len(tag_list)]

    # 태그 비율 기준 내림차순 정렬 후 상위 10개 가게 추출
    top_stores = filtered_df.sort_values("태그 비율", ascending=False).head(10)

    # 결과 저장
    recommended_stores = [
        {"가게명": row["가게명"], "유사도 점수": row["태그 비율"]}
        for _, row in top_stores.iterrows()
    ]

    final_results.append({
        "입력 텍스트": input_text,
        "추천 가게": recommended_stores,
    })

# 최종 결과 출력
for result2 in final_results:
    print(f"입력 텍스트: {result2['입력 텍스트']}")
    print("추천 가게:")
    for store in result2["추천 가게"]:
        print(f" - 가게명: {store['가게명']}, 유사도 점수: {store['유사도 점수']:.4f}")
    print()


result_df 컬럼들: Index(['입력 텍스트', '최종 선택 태그의 리스트'], dtype='object')
현재 처리 중: 퓨전식 이색 요리
입력 텍스트: 퓨전식 이색 요리
추천 가게:



<ipython-input-14-92e169b267b6>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["태그 비율"] = df[tag_list].sum(axis=1) / len(tag_list)


# TF - IDF 리뷰 유사도 추출

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

input_txt =['퓨전식 이색 요리']

if selected_category == "식당":
    df_t= pd.read_csv('/content/drive/MyDrive/result_meal.csv')
else:
    df_t= pd.read_csv('/content/drive/MyDrive/result_cafe.csv')

# 가게명이 포함된 리스트가 ranked_results['가게명']에 있다고 가정
ranked_store_names = ranked_results['가게명']

# df_t에서 store_name이 ranked_store_names에 존재하는 경우만 필터링
df_t = df_t[df_t['store_name'].isin(ranked_store_names)]

# NaN 값 제거 또는 대체
df_t["text_review"] = df_t["text_review"].fillna("")  # NaN 값을 빈 문자열로 대체

# TF-IDF 벡터라이저 초기화
tfidf_vectorizer = TfidfVectorizer()

# 데이터셋 리뷰 벡터화
corpus = df_t["text_review"].tolist()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 결과 저장을 위한 리스트
result_data = []

# 각 입력 텍스트에 대해 유사도 계산
for user_input in input_txt:
    # 입력 텍스트 벡터화
    input_vector = tfidf_vectorizer.transform([user_input])

    # 코사인 유사도 계산
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

    # 유사도 추가
    df_t["Similarity"] = similarity_scores.flatten()

    # 장소별 유사도 상위 5개의 점수 평균 계산
    top5_avg_similarity = (
        df_t.groupby("store_name")
        .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())
        .reset_index(name="Avg_Similarity")
    )

    # 평균 유사도를 기준으로 상위 5개의 장소
    top5_places = top5_avg_similarity.sort_values(by="Avg_Similarity", ascending=False).head(10)

    # 결과 저장
    result_data.append({"입력 텍스트": user_input, "상위 5개 장소 및 평균 유사도": top5_places.to_dict(orient="records")})

# 결과 데이터프레임 생성
result_df_2 = pd.DataFrame(result_data)

# 출력
print(result_df_2)

      입력 텍스트                                  상위 5개 장소 및 평균 유사도
0  퓨전식 이색 요리  [{'store_name': '구복만두', 'Avg_Similarity': 0.0}...


<ipython-input-17-78665b764df6>:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())


In [18]:
# 유사도 점수가 높은 10개의 가게 선정 및 차등 가중치 적용
final_results = []

# 각 입력 텍스트에 대해 상위 10개 가게 선정 및 점수 계산
for result in result_data:
    user_input = result["입력 텍스트"]
    top_places = pd.DataFrame(result["상위 5개 장소 및 평균 유사도"])

    # 동일한 가게가 여러 입력 텍스트에 등장할 수 있으므로 모든 결과 합치기
    if not top_places.empty:
        # 상위 10개의 가게 및 평균 유사도 추출
        sorted_places = top_places.sort_values("Avg_Similarity", ascending=False).head(10)

        # 차등 가중치 계산: 10~5에서 0.5씩 감소
        weights = [200 - i * 10 for i in range(len(sorted_places))]

        # 최종 점수 계산: 유사도 × 가중치
        sorted_places["Weighted_Score"] = sorted_places["Avg_Similarity"] * weights

        # 최종 점수 기준으로 정렬
        sorted_places = sorted_places.sort_values("Weighted_Score", ascending=False)

        # 결과 저장
        final_results.append({
            "입력 텍스트": user_input,
            "추천 가게": sorted_places[["store_name", "Weighted_Score"]].to_dict(orient="records")
        })

# 최종 결과 출력
for result in final_results:
    print(f"입력 텍스트: {result['입력 텍스트']}")
    print("추천 가게:")
    for store in result["추천 가게"]:
        print(f" - 가게명: {store['store_name']}, 점수: {store['Weighted_Score']:.2f}")
    print()

입력 텍스트: 퓨전식 이색 요리
추천 가게:
 - 가게명: 구복만두, 점수: 0.00
 - 가게명: 남산드럼통 본점, 점수: 0.00
 - 가게명: 남영탉, 점수: 0.00
 - 가게명: 만나칼국수, 점수: 0.00
 - 가게명: 샤브로21 숙명여대, 점수: 0.00
 - 가게명: 오복함흥냉면, 점수: 0.00
 - 가게명: 한강로칼국수, 점수: 0.00
 - 가게명: 한입소반, 점수: 0.00



In [19]:
# 입력 텍스트별 최종 유사도 점수 계산
final_results_by_input = {}

# result와 result2의 점수 합산
for result, result2 in zip(final_results, final_results):
    input_text = result["입력 텍스트"]

    # 입력 텍스트에 대한 각 가게의 점수 합산
    store_scores = {}

    # result에서 점수 합산
    for store in result["추천 가게"]:
        store_name = store.get("store_name", store.get("가게명"))
        score = store.get("유사도 점수", store.get("Weighted_Score"))
        if store_name and score:
            if store_name not in store_scores:
                store_scores[store_name] = 0
            store_scores[store_name] += score

    # result2에서 점수 합산
    for store in result2["추천 가게"]:
        store_name = store.get("store_name", store.get("가게명"))
        score = store.get("유사도 점수", store.get("Weighted_Score"))
        if store_name and score:
            if store_name not in store_scores:
                store_scores[store_name] = 0
            store_scores[store_name] += score

    # 입력 텍스트별로 상위 5곳 추출
    sorted_stores = sorted(store_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    final_results_by_input[input_text] = sorted_stores

# 결과 출력
for input_text, top_stores in final_results_by_input.items():
    print(f"입력 텍스트: {input_text}")
    print("최종 유사도 점수 상위 5곳:")
    for store_name, score in top_stores:
        print(f" - 가게명: {store_name}, 최종 점수: {score:.4f}")
    print()

입력 텍스트: 퓨전식 이색 요리
최종 유사도 점수 상위 5곳:

